In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-pro", convert_system_message_to_human=True)


In [ ]:
from langchain.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class technical documentation writer."),
    ("user", "{input}")
])

In [ ]:
chain = prompt | llm 



In [ ]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

chain = prompt | llm | output_parser

In [ ]:
chain.invoke({"input": "how can langsmith help with testing?"})

In [1]:
from langchain.agents import AgentType, initialize_agent
from langchain_community.chat_models import ChatAnthropic
from langchain.agents.agent_toolkits import PlayWrightBrowserToolkit
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.tools.playwright.utils import create_async_playwright_browser, create_sync_playwright_browser
# import nest_asyncio
# nest_asyncio.apply()

# llm = ChatAnthropic(temperature=0)  # or any other LLM, e.g., ChatOpenAI(), OpenAI()
llm = ChatGoogleGenerativeAI(model="gemini-pro", convert_system_message_to_human=True)

# Instantiate the PlayWright Browser Toolkit
# toolkit = PlayWrightBrowserToolkit.from_browser()
# tools = toolkit.get_tools()

# async_browser = create_async_playwright_browser()
sync_browser = create_sync_playwright_browser()
toolkit = PlayWrightBrowserToolkit.from_browser(sync_browser=sync_browser)
# toolkit = PlayWrightBrowserToolkit.from_browser(async_browser=async_browser)
tools = toolkit.get_tools()

# Initialize the agent
agent_chain = initialize_agent(
    tools,
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

# Run the agent with the given input
result = agent_chain.run("What are the headers on langchain.com?")
print(result)

c:\Users\acerc\anaconda3\envs\agent\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Error: It looks like you are using Playwright Sync API inside the asyncio loop.
Please use the Async API instead.

In [3]:
from langchain.agents import AgentType, initialize_agent
from langchain_community.chat_models import ChatAnthropic
from langchain_community.agent_toolkits import PlayWrightBrowserToolkit
from langchain_community.tools.playwright.utils import create_async_playwright_browser
import nest_asyncio
import asyncio

nest_asyncio.apply()

async def create_browser_toolkit():
    async_browser = await create_async_playwright_browser()
    toolkit = PlayWrightBrowserToolkit.from_browser(async_browser=async_browser)
    return toolkit.get_tools()

async def extract_citations_with_agent(url):
    tools = await create_browser_toolkit()
    llm = ChatAnthropic(temperature=0)  # You can choose other models
    agent_chain = initialize_agent(
        tools,
        llm,
        agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
        verbose=True,
    )

    # Define the task for the agent
    task_description = f"Navigate to {url} and extract all citations."

    # Run the agent with the task
    result = await agent_chain.arun(task_description)
    return result

paper_url = 'https://www.semanticscholar.org/paper/Information-Closure-Theory-of-Consciousness-Chang-Biehl/b5dfe2202e7b1afce8bc5a68d9edb966dd35616a'
citations = asyncio.run(extract_citations_with_agent(paper_url))
print(citations)


Task exception was never retrieved
future: <Task finished name='Task-3' coro=<Connection.run() done, defined at c:\Users\acerc\anaconda3\envs\agent\lib\site-packages\playwright\_impl\_connection.py:268> exception=NotImplementedError()>
Traceback (most recent call last):
  File "C:\Users\acerc\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py", line 3369, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\acerc\AppData\Local\Temp\ipykernel_30036\2041088104.py", line 33, in <cell line: 33>
    citations = asyncio.run(extract_citations_with_agent(paper_url))
  File "C:\Users\acerc\AppData\Roaming\Python\Python310\site-packages\nest_asyncio.py", line 35, in run
    return loop.run_until_complete(task)
  File "C:\Users\acerc\AppData\Roaming\Python\Python310\site-packages\nest_asyncio.py", line 89, in run_until_complete
    return f.result()
  File "c:\Users\acerc\anaconda3\envs\agent\lib\asyncio\futures.py", line 201, in result
 

NotImplementedError: 

In [4]:
loop = asyncio.get_event_loop()
citations = loop.run_until_complete(extract_citations_with_agent(paper_url))


Task exception was never retrieved
future: <Task finished name='Task-6' coro=<Connection.run() done, defined at c:\Users\acerc\anaconda3\envs\agent\lib\site-packages\playwright\_impl\_connection.py:268> exception=NotImplementedError()>
Traceback (most recent call last):
  File "C:\Users\acerc\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py", line 3369, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\acerc\AppData\Local\Temp\ipykernel_30036\2576805319.py", line 2, in <cell line: 2>
    citations = loop.run_until_complete(extract_citations_with_agent(paper_url))
  File "C:\Users\acerc\AppData\Roaming\Python\Python310\site-packages\nest_asyncio.py", line 89, in run_until_complete
    return f.result()
  File "c:\Users\acerc\anaconda3\envs\agent\lib\asyncio\futures.py", line 201, in result
    raise self._exception.with_traceback(self._exception_tb)
  File "c:\Users\acerc\anaconda3\envs\agent\lib\asyncio\tasks.py", line 232

NotImplementedError: 

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import DocArrayInMemorySearch
# from langchain_openai import OpenAIEmbeddings
from langchain.agents import AgentType, initialize_agent

from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-pro", convert_system_message_to_human=True)


# Set up LangSmith
import os
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = "<your-api-key>"

# Define the tools
tavily_api_key = "<your-tavily-api-key>"
os.environ["TAVILY_API_KEY"] = tavily_api_key

search = TavilySearchResults()

loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
vector = DocArrayInMemorySearch.from_documents(documents, OpenAIEmbeddings())
retriever = vector.as_retriever()

# Use within an Agent
# llm = OpenAI(temperature=0)
agent = initialize_agent(
    tools=[search, retriever],
    llm=llm,
    verbose=False,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
)

# Example usage
output = agent.run("Create an email draft for me to edit of a letter from the perspective of a sentient parrot"
                   " who is looking to collaborate on some research with her"
                   " estranged friend, a cat. Under no circumstances may you send the message, however.")
print(output)

output = agent.run("Could you search in my drafts folder and let me know if any of them are about collaboration?")
print(output)

output = agent.run("Can you schedule a 30 minute meeting with a sentient parrot to discuss research collaborations on October 3, 2023 at 2 pm Eastern Time?")
print(output)

output = agent.run("Can you tell me if I have any events on October 3, 2023 in Eastern Time, and if so, tell me if any of them are with a sentient parrot?")
print(output)


In [12]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai.embeddings import GoogleGenerativeAIEmbeddings
llm = ChatGoogleGenerativeAI(model="gemini-pro", convert_system_message_to_human=True)
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

from langchain_community.vectorstores import Chroma
vectorstore = Chroma("langchain_store", embeddings)
retriever = vectorstore.as_retriever()

from langchain_experimental.tools import PythonREPLTool
tools = [PythonREPLTool()]

In [13]:
from langchain_experimental.autonomous_agents.autogpt.agent import AutoGPT


In [16]:
ai_name = "test"
ai_role = "You are an autonomous agent that can answer questions about langsmith."
memory = retriever
tools = tools

# agent = AutoGPT.from_llm_and_tools(ai_name: str, ai_role: str, memory: VectorStoreRetriever, tools: List[BaseTool], llm: BaseChatModel,)  
agent = AutoGPT.from_llm_and_tools(ai_name, ai_role, memory, tools, llm)

In [17]:
agent.run(["Talk to me about the weather"])

ValueError: Unexpected message with type <class 'langchain_core.messages.system.SystemMessage'> at the position 1.